To Do:
Clean up result and separate into result, score, and opponent score
Convert Game type to words

In [1]:
import os
import pandas as pd
import sqlalchemy
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

connection_string = os.getenv("LINODE_CONN_STRING")
connection_string = os.getenv("LINODE_CONN_STRING")
conn = sqlalchemy.create_engine(os.getenv("LINODE_CONN_STRING")).connect()

In [2]:
data = pd.read_sql("SELECT * FROM sports_data.game_stats", conn)

In [3]:
data['Win_Loss'] = data['result'].apply(lambda x: x[:1])

In [4]:
data['Win_Loss'].value_counts()

Win_Loss
W    53759
L    53759
Name: count, dtype: int64

In [5]:
def determine_scores(row):
    if row['Win_Loss'] == 'W':
        row['score'] = row['result'][3:].split('-')[0]
        row['opponent_score'] = row['result'][3:].split('-')[1]
    else:
        row['score'] = row['result'][3:].split('-')[1]
        row['opponent_score'] = row['result'][3:].split('-')[0]
    return row

In [6]:
data = data.apply(determine_scores, axis=1)

In [7]:
data['opponent_score'] = data['result'].apply(lambda x: x[3:].split('-')[1])

In [8]:
data[['result', 'Win_Loss', 'score', 'opponent_score']]

,result,Win_Loss,score,opponent_score
0,"W, 87-71",W,87,71
1,"L, 87-71",L,71,71
2,"L, 69-62",L,62,62
3,"W, 69-62",W,69,62
4,"W, 65-59",W,65,59
...,...,...,...,...
107513,"W, 71-57",W,71,57
107514,"L, 72-68",L,68,68
107515,"W, 72-68",W,72,68
107516,"L, 88-77",L,77,77


In [9]:
for i, row in data.iterrows():
    score = int(row['score'])
    opp_score = int(row['opponent_score'])
    if row['Win_Loss'] == 'W' and opp_score > score:
        print(f"Error at index {i}")
    elif row['Win_Loss'] == 'L' and score > opp_score:
        print(f"Error at index {i}")

In [10]:
# 0 is Non-Conference, 1 is conference, 2 is conference tourney, 3 is NCAA tourney (generally? might be exceptions where 3 is other post season tourney)
def convert_game_type(row):
    if row['game_type'] == 0:
        return 'Non-Conference'
    elif row['game_type'] == 1:
        return 'Conference'
    elif row['game_type'] == 2:
        return 'Conference Tourney'
    elif row['game_type'] == 3:
        return 'Post-Season Tourney'
    else:
        return 'Other'

In [11]:
data['game_type'] = data.apply(convert_game_type, axis=1)

In [12]:
data['game_type'].value_counts()

game_type
Conference             61980
Non-Conference         37870
Conference Tourney      5592
Post-Season Tourney     2076
Name: count, dtype: int64

In [13]:
data.shape[0]

107518

In [15]:
conn.rollback()

In [16]:
data.to_sql('game_stats_v3', sqlalchemy.create_engine(os.getenv("LINODE_CONN_STRING")), if_exists='replace', index=False, schema='sports_data')

518